## Шаг 1. Предобработка данных

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats as st
import numpy as np
import math as mth

import warnings

warnings.filterwarnings("ignore")

In [ ]:
data_users = pd.read_csv('/datasets/user_source.csv')
display(data_users.head(5), data_users.info(), data_users.isnull().sum(), data_users.duplicated().sum())

print(data_users['source'].unique())

In [ ]:
data_game_act = pd.read_csv('/datasets/game_actions.csv')
display(data_game_act.head(5), data_game_act.info(), data_game_act.isnull().sum(), data_game_act.duplicated().sum())

In [ ]:
print(data_game_act['event'].unique())
print() 
print(data_game_act['building_type'].unique())
print()
print(data_game_act['project_type'].unique())
print()
print(data_game_act['event_datetime'].min())
print()
print(data_game_act['event_datetime'].max())

Пропуски в building_type и project_type оставляем

In [ ]:
#Переводим в формат даты колонку event_datetime
data_game_act['event_datetime'] = pd.to_datetime(data_game_act['event_datetime'])
#Удаляем 1 дубль
data_game_act = data_game_act.drop_duplicates().reset_index(drop=True)

In [ ]:
data_ad_сost = pd.read_csv('/datasets/ad_costs.csv')
display(data_ad_сost.head(5), data_ad_сost.info(), data_ad_сost.isnull().sum(), data_ad_сost.duplicated().sum())

Пропуски в building_type и project_type оставляем, также был удален дубликат из data_game_act

______

## Шаг 2. Exploratory data analysis

### Сравним количество пользователей, завершивших 1 уровень, PvP и PvE

In [ ]:
print('Всего в когорте', data_game_act['user_id'].nunique(), 'уникальных пользователей')

____

#### Группа PVE

Посчитаем сколько зданий строят игроки PVE

In [ ]:
data_pve_id = data_game_act[(data_game_act['event'] == 'project')]

data_finish = data_game_act[(data_game_act['event'] == 'finished_stage_1')]

data_pve =  data_finish.merge(data_pve_id['user_id'], on='user_id', how='inner')

print(data_pve['user_id'].nunique(), 'пользователей завершили 1 уровень, завершив проект')

In [ ]:
data_pve = data_game_act.merge(data_pve['user_id'], on = 'user_id', how='right')

In [ ]:
data_pve_agg = data_pve.query('event == "building"').groupby('user_id').agg({'event': 'count'}).reset_index()
data_pve_agg.columns = ['user_id', 'count_build']
data_pve_agg['strategy'] = 'PVE'
print(round(data_pve_agg['count_build'].mean(), 2), 'среднее количество зданий на пользователя PVE')
print(data_pve_agg['count_build'].min(), 'минимальное количество зданий на пользователя PVE')
print(data_pve_agg['count_build'].max(), 'максимальное количество зданий на пользователя PVE')
display(data_pve_agg)

In [ ]:
fig, hist = plt.subplots()
hist = sns.histplot(data=data_pve_agg, x="count_build")
hist.set_title('количество построек PVE') 
fig.set_figwidth(10)
fig.set_figheight(8);

____

#### Группа PVP

Посчитаем сколько зданий строяк игроки PVP

In [ ]:
# выбираем пользователей, которые завершили уровень без проекта
data_pvp = pd.concat([data_game_act, data_pve]).drop_duplicates(keep=False) #объединяем таблицы и удаляем дубли, чтобы найти пользователей не их группы pve
data_pvp = data_pvp.loc[(data_game_act['event'] == 'finished_stage_1')] #берем тех, кто закончил

display(data_pvp['user_id'].nunique())

In [ ]:
data_pvp = data_game_act.merge(data_pvp['user_id'], on='user_id', how='inner')

In [ ]:
data_pvp = data_pvp.query('event == "building"').groupby('user_id').agg({'event': 'count'}).reset_index()
data_pvp.columns = ['user_id', 'count_build']
data_pvp['strategy'] = 'PVP'
print(round(data_pvp['count_build'].mean(), 2), 'среднее количество зданий на пользователя PVP')
print(data_pvp['count_build'].min(), 'минимальное количество зданий на пользователя PVP')
print(data_pvp['count_build'].max(), 'максимальное количество зданий на пользователя PVP')

In [ ]:
fig, hist = plt.subplots()
hist = sns.histplot(data=data_pvp, x="count_build")
hist.set_title('количество построек PVP') 
fig.set_figwidth(10)
fig.set_figheight(8);

Пользователи PVP в среднем строят на 3 здания меньше, но надо учесть, что их почти в 2 раза больше, чем пользователей PVE

### Общее количество зданий

In [ ]:
print(data_game_act[data_game_act['event'] == 'building']['event'].count(), 'всего было построено зданий')

### Среднее количество построек на пользователя 

In [ ]:
print(round(data_game_act.query('event == "building"').groupby('user_id').agg({'event': 'count'}).median(), 2), 'среднее количество зданий на пользователя')

### Периодичность построек

In [ ]:
data_act_buil = data_game_act.query('event == "building"')

data_act_buil['prev'] = (data_act_buil.query('event == "building"').sort_values(by=['event_datetime'], ascending=True)
                       .groupby(['user_id'])['event_datetime'].shift(1))

data_act_buil['diff'] = data_act_buil['event_datetime'] - data_act_buil['prev']

print('в среднем времени между каждой постройков проходит', data_act_buil['diff'].median()) #используем медиану, чтобы избежать выбросов

In [ ]:
day_f = data_game_act.query('event == "building"').groupby('user_id').apply(lambda x: x['event_datetime'].max() - x['event_datetime'].min()).dt.round('D').reset_index()
day_f.columns = ['user_id', 'time']
day_f['time'] = (day_f['time']/ np.timedelta64(1, 'D')).astype(int) #переводим в формат int
print('от первой постройки до последней в среднем проходит', day_f['time'].median(), 'дней')

In [ ]:
fig, hist = plt.subplots()
hist = sns.histplot(data=day_f, x="time")
hist.set_title('количество дней стройки') 
fig.set_figwidth(10)
fig.set_figheight(8);

###  Проанализировать влияние событий на совершение целевого события (переход на 2 уровень)

Разделим пользователей на, завершивших уровень (1) и не завершивших (0), и сверим их показатели

In [ ]:
#создаем df с пользователями, которые завершили уровень
data_finished = data_game_act.merge(data_finish['user_id'], on = 'user_id', how='right')
display(data_finished['user_id'].nunique())

#создаем df с пользователями, которые не завершили уровень
data_not_finished = pd.concat([data_game_act, data_finished]).drop_duplicates(keep=False)
display(data_not_finished['user_id'].nunique())

#### среднее количество времени, проведенное пользователем на 1 уровне

In [ ]:
day_to_f = data_finished.query('event == "building"').groupby('user_id').apply(lambda x: x['event_datetime'].max() - x['event_datetime'].min()).dt.round('D').reset_index()
day_to_f.columns = ['user_id', 'day_to_finish']
day_to_f['day_to_finish'] = (day_to_f['day_to_finish']/ np.timedelta64(1, 'D')).astype(int) #переводим в формат int

sns.boxplot(y='day_to_finish', data=day_to_f)
plt.ylim(0, 30);

day_to_f = day_to_f[day_to_f['day_to_finish'] < 20]
print('от первой постройки до последней в среднем проходит', day_to_f['day_to_finish'].mean(), 'дней')

In [ ]:
fig, hist = plt.subplots()
hist = sns.histplot(data=day_to_f, x="day_to_finish", bins = 20)
hist.set_title('количество дней стройки') 
fig.set_figwidth(10)
fig.set_figheight(8);

In [ ]:
day_to_end = data_not_finished.query('event == "building"').groupby('user_id').apply(lambda x: x['event_datetime'].max() - x['event_datetime'].min()).dt.round('D').reset_index()
day_to_end.columns = ['user_id', 'time']
day_to_end['time'] = (day_to_end['time']/ np.timedelta64(1, 'D')).astype(int) #переводим в формат int

sns.boxplot(y='time', data=day_to_end)
plt.ylim(0, 50);

day_to_end = day_to_end[day_to_end['time'] < 25]
print('от первой постройки до прекращения игры в среднем проходит', day_to_end['time'].mean(), 'дней')

In [ ]:
fig, hist = plt.subplots()
hist = sns.histplot(data=day_to_end, x="time")
hist.set_title('количество дней игры до конца') 
fig.set_figwidth(10)
fig.set_figheight(8);

у большинства пользователей заканчивается терпение с 6-11 день, именно в это время др пользователи переходили на 2 уровень 

#### скорость постройки

In [ ]:
data_act_buil_1 = data_finished.query('event == "building"')

data_act_buil_1['prev'] = (data_act_buil_1.query('event == "building"').sort_values(by=['event_datetime'], ascending=True)
                       .groupby(['user_id'])['event_datetime'].shift(1))

data_act_buil_1['diff'] = data_act_buil_1['event_datetime'] - data_act_buil_1['prev']

In [ ]:
diff_buil_1 = data_act_buil_1['diff'].dropna() #удаляем пустые строки, чтобы перевести в часы
diff_buil_1 = data_act_buil_1.merge(diff_buil_1, on = 'diff', how = 'inner').drop_duplicates()
diff_buil_1['diff'] = (diff_buil_1['diff']/ np.timedelta64(1, 'h')).astype(int)

display(diff_buil_1.head(5))

In [ ]:
#посчитаем для каждого пользователя среднее время между постройками
time_diff_mean = diff_buil_1.groupby('user_id').agg({'diff':'mean'}).reset_index()
time_diff_mean.columns = ['user_id', 'mean_time_building']

In [ ]:
sns.boxplot(y='mean_time_building', data=time_diff_mean)
plt.title('среднее время между постройками группы 1 (ч)')
plt.ylim(0, 80);

In [ ]:
a = time_diff_mean[time_diff_mean['mean_time_building'] < 60]

In [ ]:
fig, hist = plt.subplots()
hist = sns.histplot(data=a, x="mean_time_building")
hist.set_title('количество часов между постройками 1') 
fig.set_figwidth(10)
fig.set_figheight(8);

In [ ]:
print(' у группы 1 в среднем между каждой постройкой проходит', a['mean_time_building'].mean().round(2), 'часов')

In [ ]:
data_act_buil = data_not_finished.query('event == "building"')

data_act_buil['prev'] = (data_act_buil.query('event == "building"').sort_values(by=['event_datetime'], ascending=True)
                       .groupby(['user_id'])['event_datetime'].shift(1))

data_act_buil['diff'] = data_act_buil['event_datetime'] - data_act_buil['prev']

In [ ]:
diff_buil = data_act_buil['diff'].dropna() #удаляем пустые строки, чтобы перевести в часы
diff_buil = data_act_buil.merge(diff_buil, on = 'diff', how = 'inner').drop_duplicates()
diff_buil['diff'] = (diff_buil['diff']/ np.timedelta64(1, 'h')).astype(int)

display(diff_buil.head(5))

In [ ]:
time_diff_mean_0 = diff_buil.groupby('user_id').agg({'diff':'mean'}).reset_index()
time_diff_mean_0.columns = ['user_id', 'mean_time_building']

sns.boxplot(y='mean_time_building', data=time_diff_mean_0)
plt.title('среднее время между постройками группы 0(ч)')
plt.ylim(0, 80);

In [ ]:
b = time_diff_mean_0[time_diff_mean_0['mean_time_building'] < 80]

print('в среднем у группы 0 времени между каждой постройков проходит', b['mean_time_building'].mean().round(2), 'часов')

In [ ]:
fig, hist = plt.subplots()
hist = sns.histplot(data=b, x="mean_time_building")
hist.set_title('количество часов между постройками 0') 
fig.set_figwidth(10)
fig.set_figheight(8);

#### количество построек

In [ ]:
total_1 = pd.concat([data_pve_agg, data_pvp])
print('Среднее количество построек у группы 1:', total_1['count_build'].median().round(2))

fig, hist = plt.subplots()
hist = sns.histplot(data=total_1, x="count_build")
hist.set_title('количество построек группа 1') 
fig.set_figwidth(10)
fig.set_figheight(8);

In [ ]:
bild_0 = data_not_finished.query('event == "building"').groupby('user_id').agg({'event_datetime':'count'}).reset_index()
bild_0.columns = ['user_id', 'cnt_buil']
print('Среднее количество построек у группы 0:', bild_0['cnt_buil'].median().round(2))
display(bild_0)
fig, hist = plt.subplots()
hist = sns.histplot(data=bild_0, x="cnt_buil", bins = 20)
hist.set_title('количество построек группа 0') 
fig.set_figwidth(10)
fig.set_figheight(8);

**Создадим таблицу со всей нашей подготовленной инф**

In [ ]:
total_1 = total_1.merge(day_to_f, on = 'user_id', how = 'inner')

total_1 = total_1.merge(a, on = 'user_id', how = 'inner')

display(total_1)

In [ ]:
total_0 = data_not_finished['user_id'].drop_duplicates()
total_0 = day_to_end.merge(total_0, on = 'user_id', how = 'inner')
total_0 = total_0.merge(b, on = 'user_id', how = 'inner')
total_0 = total_0.merge(bild_0, on = 'user_id', how = 'inner')
display(total_0)

#### канал привлечения

In [ ]:
#добавим канал привлечения в таблицу
total_1 = total_1.merge(data_users, on = 'user_id', how = 'inner')

total_0 = total_0.merge(data_users, on = 'user_id', how = 'inner')

In [ ]:
total_1.groupby('source', as_index = False).agg({'user_id':'count'})

In [ ]:
total_0.groupby('source', as_index = False).agg({'user_id':'count'})

преданность игре не зависит от источника пользователя

## Проверка гипотез

H0 - Различие среднего времени прохождения 1 уровня между пользователями стратегии PvP и PvE отсутствует

H1 - Различие среднего времени прохождения 1 уровня между пользователями стратегии PvP и PvE есть

In [ ]:
a = total_1[total_1['strategy'] == 'PVE']['day_to_finish']
b = total_1[total_1['strategy'] == 'PVP']['day_to_finish']

alpha = .05
result = st.ttest_ind(a, b, equal_var=False)
print('p-значение:', result.pvalue)

if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

__________

H0 - Среднее время между постройками не влияет на завершение 1 уровня

H1 - Среднее время между постройками влияет на завершение 1 уровня


In [ ]:
a = total_1['mean_time_building']
b = total_0['mean_time_building']

alpha = .05
result = st.ttest_ind(a, b, equal_var=False)
print('p-значение:', result.pvalue)

if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

____
H0 - Среднее количество построек между PVP и PVE одинаково

H1 - Среднее количество построек между PVP и PVE различно

In [ ]:
a = total_1['count_build']
b = total_0['cnt_buil']

alpha = .05
result = st.ttest_ind(a, b, equal_var=False)
print('p-значение:', result.pvalue)

if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

что мы имеем, кол-во построек, скороcть прохождения уровня у двух групп различается, среднее время постройки влияет на завершение 1 уровня

_____
## Проверка модели монетизации

один показ рекламы приносит 0.07 у.е

нам известно, что было решено показывать рекламу на экране с выбором постройки

всего было построено 127956 зданий

In [ ]:
#Общая сумма затрат на рекламу игры

data_ad_сost['cost'].sum()

#Один показ рекламы приносит 0.07 у.е.

print(round(data_ad_сost['cost'].sum()/0.07, 0), 'нужно показов, чтобы окупить затраты на рекламу')

с текущей моделью затраты на маркетинг окупаются, но можно снизить рекламную нагрузку. Надо еще учесть, что пользователи могли открыть экран с постройкой и до завершения предыдущей. 

Сейчас пользователям показывают рекламу в среднем 10 раз (среднее кол-во построек на пользователя) раз в 20 часов (среднее время между постройками). всего пользователи проводят 240 часов в игре.

1. Предлагаю отвязаться от показа рекламы на экране выбора постройки. И перейти к показу рекламы по времени.

В среднем пользователи строят новое здание раз в 20 часов и играют 10 дней.

Если показывать рекламу раз в сутки, то прибыль = 135760 уе.
Тем самым снизим рекламную нагрузку на пользователей (экран выбора постройки могли открывать и до завершения предыдущей, т.е. до возможности стройки).
И снизим риск снижения дохода при большем количестве пользователей PVP.

In [ ]:
data_game_act.groupby('building_type').agg({'user_id':'count'})

In [ ]:
data_game_act = data_game_act.sort_values(by = ['user_id', 'event_datetime'])
first_buil = data_game_act.groupby('user_id').agg({'event_datetime': 'min', 'building_type': 'first'}).reset_index()

display(first_buil.groupby('building_type').agg({'user_id':'count'}))

In [ ]:
first_diff = pd.concat([diff_buil_1, diff_buil]).sort_values(by = ['user_id', 'event_datetime'])
first_diff = first_diff.groupby('user_id').agg({'diff': 'first'}).reset_index()
display(first_diff.query('diff < 20').count())

2. Пропуск рекламы на первых зданиях.

первая постройка у всех assembly_shop. в среднем можно снизить кол-во рекламы на 1,4 показа на пользователя. 
например, у пользователей, которые чаще заходят в игру можно сделать первые 2 постройки без рекламы(в нашей когорте это 5968 пользователей), у др 1. Т.к. пользователи, которые заходят в игру чаще, вероятнее закончат 1 уровень. 


Дополнительные идеи:

1 Интегрировать рекламу в интерфейс игры (формы зданий, кораблей,
надписи, окраска)

2 Показ рекламы для ускорения постройки (Rewarded video ads). Может
способствовать удержанию пользователей в игре, так и увеличить кол-во пользователей PVE